<a href="https://colab.research.google.com/github/sinjy1203/deep-learning/blob/master/dacon_universe(final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

train = pd.read_csv('/content/gdrive/My Drive/dacon stage 2 data/train.csv', index_col=0)
test = pd.read_csv('/content/gdrive/My Drive/dacon stage 2 data/test.csv', index_col=0)
sample_submission = pd.read_csv('/content/gdrive/My Drive/dacon stage 2 data/sample_submission.csv', index_col=0)
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i

def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))
#######
index = train.columns
index_list = []
for i in index:
  index_list.append(i)
index_list = np.array(index_list)  
index_list_drop = index_list[(index_list != 'type')]
index_list_drop2 = index_list_drop[(index_list_drop != 'type_num')]
index_list_drop3 = index_list_drop2[(index_list_drop2 != 'fiberID')]
index_list_drop3 = [x for x in index_list_drop3]
std = train.std(0)
for i, name in enumerate(index_list_drop3):
  train = train[train[name] >= train[name].mean(0) -2 * std[name]]
  train = train[train[name] <= train[name].mean(0) +2 * std[name]]
train_x = train.drop(columns=['type', 'type_num'], axis=1)
train_y = train['type_num']
test_x = test
cat_x = train_x["fiberID"]
train_x = train_x.drop(columns='fiberID', axis=1)
std_scaler = StandardScaler()
train_x = std_scaler.fit_transform(train_x)
train_x = pd.DataFrame(data=train_x, columns=index_list_drop3)
# plt.figure(figsize=(15, 10))
# train_x.boxplot(column=index_list_drop3)
######

cat_x_encoded, cat_x_categories = cat_x.factorize()
encoder = OneHotEncoder(categories='auto')
cat_x_1hot = encoder.fit_transform(cat_x_encoded.reshape(-1,1))
cat_x_1hot_array = cat_x_1hot.toarray()

combine_train_x = np.c_[train_x, cat_x_1hot_array]
x_train = combine_train_x
print(x_train[:3])

cat_test_x = test_x["fiberID"]
cat_test_x_encoded, cat_test_x_categories = cat_test_x.factorize()
cat_test_x_1hot = encoder.transform(cat_test_x_encoded.reshape(-1,1))
cat_test_x_1hot_array = cat_test_x_1hot.toarray()
drop_test_x1 = test_x.drop("fiberID", axis=1)
drop_test_x = std_scaler.transform(drop_test_x1)
combine_test_x = np.c_[drop_test_x, cat_test_x_1hot_array]
x_test = combine_test_x
print(len(x_train))

tf.set_random_seed(777)
y_train = train_y
y_train = np.array(y_train)
y_train = y_train.reshape(-1, 1)

x_train1 = x_train[:199000]
x_train2 = x_train[199000:]
y_train1 = y_train[:199000]
y_train2 = y_train[199000:]

x = tf.placeholder(tf.float32, [None, 1020])
y = tf.placeholder(tf.int32, [None, 1])
keep_prob = tf.placeholder(tf.float32)
y_one_hot = tf.one_hot(y, 19)
y_one_hot = tf.reshape(y_one_hot, [-1, 19])

W1 = tf.get_variable("w11111111", shape=[1020, 1054], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([1054]))
L1 = tf.nn.leaky_relu(tf.matmul(x, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob= keep_prob)

W2 = tf.get_variable("w22222222", shape=[1054, 1054], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([1054]))
L2 = tf.nn.leaky_relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob= keep_prob)

W3 = tf.get_variable("w33333333", shape=[1054, 1054], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([1054]))
L3 = tf.nn.leaky_relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob= keep_prob)

W4 = tf.get_variable("w44444444", shape=[1054, 1054], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([1054]))
L4 = tf.nn.leaky_relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob= keep_prob)

W5 = tf.get_variable("w55555555", shape=[1054, 512], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([512]))
L5 = tf.nn.leaky_relu(tf.matmul(L4, W5) + b5)
L5 = tf.nn.dropout(L5, keep_prob= keep_prob)

W6 = tf.get_variable("w66666666", shape=[512, 512], initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.Variable(tf.random_normal([512]))
L6 = tf.nn.leaky_relu(tf.matmul(L5, W6) + b6)
L6 = tf.nn.dropout(L6, keep_prob= keep_prob)

W7 = tf.get_variable("w77777777",shape=[512, 19], initializer=tf.contrib.layers.xavier_initializer())
b7 = tf.Variable(tf.random_normal([19]))
logits = tf.matmul(L6, W7) + b7
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_one_hot))
train = tf.train.AdamOptimizer(0.005).minimize(cost)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(hypothesis,1), tf.argmax(y_one_hot,1)), tf.float32))

total_epochs = 10
batch_size = 1000
number = int(199000 / batch_size)
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for epochs in range(total_epochs):
    avg_cost = 0
    for i in range(number):
      xs = x_train1[i * batch_size:(i + 1) * batch_size]
      ys = y_train1[i * batch_size:(i + 1) * batch_size]
      _, cost_val = sess.run([train, cost], feed_dict={x: xs, y: ys, keep_prob: 0.5})
      if(i == (number - 1)):
        _, cost_val = sess.run([train, cost], feed_dict={x: x_train2, y: y_train2, keep_prob: 0.5})
      avg_cost += cost_val / (number + 1)
    if(epochs % 1 == 0):
      print(epochs, '  ', avg_cost)

  y_pred = sess.run(hypothesis, feed_dict={x: x_test, keep_prob: 1})
  print(y_pred)

  submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
  submission.to_csv('/content/gdrive/My Drive/submission7.csv', index=True)
  #####score = 0.88

[[ 0.78596369  0.82628111  1.17212226 ...  0.          0.
   0.        ]
 [ 0.15910134  0.44512825  0.80453329 ...  0.          0.
   0.        ]
 [-1.1110021  -1.65070398 -1.50942413 ...  0.          0.
   0.        ]]
199942
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0    6.3215809977054604
1    1.0907246562838555
2    0.9021916526556015
3    0.8063376608490944
4    0.